<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-\amily:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Data Leakage
              
</p>
</div>

Data Science Cohort Live NYC Feb 2022
<p>Phase 3: Topic 24</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>

#### Data Leakage
- When information not generally available/used at prediction time contaminates the modeling training process.

<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> The despair of realizing you have a data leak.</center>

Leads to overconfident estimates of model performance during the validation and testing phases.

- Bad performance after model deployment in production.


<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> My overlords are upset.</center>

<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> I have spent all night trying to find the data leak.</center>

<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> What is the nature of my leak?</center>

<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> Where is it??</center>

Diagnosing a data leak can be subtle business:
- Understanding different types of leak
- Where in the process they can be accidentally introduced.

#### Training leakage

Fitting and applying transformations **before** train-test(hold out) splitting.

<img src = "Images/training_testing.png" width = 500 />

- Why is this bad?

Applying transformation to training set:
- contains information from test set (contained in fitted attributes of Transformer)

- Contaminated training set.
- Model has inadvertently trained on information you are trying to predict.

#### You can introduce it by accident on really innocuous steps.

In [10]:
import pandas as pd
titanic_df = pd.read_csv('Data/titanic.csv').drop(columns = ['Cabin'])
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


Let's impute those NaNs in age with the mean.

In [13]:
# impute with the mean
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean()
                                            )
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


Bam. You just introduced data leakage.
- imputed with statistics of entire dataset before train-test split.
- Statistics of train in test and vice-versa.
- Whether impact is significant will depend on data and model.

#### Leakage between train and true test can be a truly costly mistake:
- You'll only figure it out after you've deployed
- Made suboptimal predictions:
    - incorrectly recommending inventory changes to increase sales
    - or *much* worse.


#### Leakage in cross validation

Less bad but still not good...

<img src = "Images/crossval.png" width = 500/>

When you pollute your training fold with your validation fold:

- Each cross-validation trial has data leakage.


Validation performance measurements not correct:
- Incorrect estimates of average model performance
- Incorrect estimates of model variance.

Messes up your hyperparameter tuning:
- "Best model": hyperparameter settings with best average model performance
- But it doesn't work well on my test/hold-out set...

#### Then check for data leakage in your cross-validation

#### Other types of data leakage

#### Feature leakage

Dependent on how the data/features were collected.

<img src = "Images\data_leakage_predcontam.png" width = 800/>

- Predicting whether we should approve a loan or not.
- Have database of current lendees.

- Features include:
    - Salary information at time of loan approval
    - Pre-loan bankruptcy history
    - FICO score
    - Listed occupation
    - Monthly interest payments.
    - Bank balance across accounts.
    

**What's the problem here?**

Possibly conflating information from *after* prediction with properties before prediction.
- Features from **after** prediction potentially affected by target.
- Our target is now leaking into the way we trained our model.
- Features *before* and *after* approval may not be drawn from same distribution.

**These sources of feature leakage can be very subtle**
- Understanding of the data collection process and data definitions is critical here.

**Case Study**

Predict sale price of a given home.

- Size of the house (in square meters)
- Average sales price of homes in the same neighborhood
- Latitude and longitude of the house
- Whether the house has a basement

Is there a source of potential data leakage? Explain.

Depends on whether the average includes the sales prices including given sale or before it.

**Another Case Study**

Want to predict rate of infection during surgery based on:
- patient specific factors (immuno-history, etc.)
- environmental factors (hospital cleanliness inspections, etc.)

You idea: include surgeon as a factor.

1. Take all surgeries by each surgeon and calculate the infection rate among those surgeons.
2. For each patient in the data, find out who the surgeon was and plug in that surgeon's average infection rate as a feature.

Potentially introduces both target leakage and train-test leak:

- Target leakage: if given patient's outcome contributes to the infection rate for his surgeon. 
- Using target to calculate feature.
- Then using this feature to predict target.

Can avoid by using:
- Surgeon's infection rate for only surgeries before the patient we are predicting for.
- Tricky, for sure.

Train-test contamination problem if you calculate this using all surgeries a surgeon performed
- Including those from the test-set. 

Are you tearing your hair out? Good. You now understand data leakage.